In [3]:
import numpy as np
import numpy.linalg as LA
import matplotlib.pyplot as plt
import scipy.linalg as sla

In [ ]:
# define lattice and generate hopping matrix with OBC

In [2]:
from pythtb import *
def Tmatrix(t,mu):

    # set model parameters
    mu=0.0
    t=-1.0
    lat=[[1.0,0.0],[0.0,1.0]]
    orb=[[0.0,0.0]]

    # create TB model
    my_model=tb_model(2,2,lat,orb)
    
    # set on-site energies
    my_model.set_onsite([-mu])
    # set hopping terms (one for each nearest neighbor)
    my_model.set_hop(t, 0, 0, [1, 0])
    my_model.set_hop(t, 0, 0, [0, 1])

    # create a 4x4 lattice
    Nx, Ny = 4, 4
    my_ham = my_model.cut_piece(Nx,0,glue_edgs=False)
    my_ham = my_ham.cut_piece(Ny,1,glue_edgs=False)
    # print Hamiltonian
    return my_ham._gen_ham()


In [44]:
L = 4 * 4
Ntau = 20
dtau = 0.1
U = 4.0
t = -1.0
mu = U/2
beta = dtau * Ntau
alpha = np.arccosh(np.exp(0.5*dtau*U))
T = Tmatrix(t,mu).real
ExpT = sla.expm(-dtau * T)
Vdiag = np.zeros(L)
Vdiag[:] = alpha

In [22]:
def ExpM(phi,ExpT,Vdiag):
    # T = exp(-T*dtau)
    return ExpT.dot(np.diag(phi * Vdiag))

In [52]:
def SVD_stablizer(A,U,D,V,dir = 'Forward'):
    # For forward direction
    # B = U * D * V
    # A * B = A * U * D * V = U1 * D1 * V1 * V = U1 * D1 * V2
    # For backward direction
    # B = U * D * V
    # B * A = U * D * V * B = U * U1 * D1 * V1 = U2 * D1 * V1

    if dir == 'Forward':
        U, D, V1 = LA.svd(A.dot(U).dot(np.diag(D)))
        return U, D, V1.dot(V)
    elif dir == "Backward":
        U1, D, V = LA.svd(np.diag(D).dot(V).dot(A))
        return U.dot(U1), D, V
    else:
        raise ValueError("dir must be Forward or Backward")

In [12]:
def GF(UL,DL,VL,UR,DR,VR):
    # B(t,0) = UL * DL * VL
    # B(beta,t) = VR * DR * UR Take care of it !!!
    U,D,V = LA.svd(LA.inv(UL.dot(UR))+np.diag(DR).dot(VR).dot(VL).dot(np.diag(DL)))
    return LA.inv(V.dot(UL)).dot(np.diag(1/D)).dot(UR.dot(U))

In [39]:
def Stable_Bmatrix(phi,ExpT,Vdiag):
    # use SVD algorithm to calculate the stable B matrix
    # ExpT: the matrix of exp(-dtau * T)
    # Vdiag: Ntau x L matrix with each column being - alpha 
    Ntau, L = phi.shape
    U = np.zeros((Ntau+1,L,L))
    D = np.zeros((Ntau+1,L))
    V = np.zeros((Ntau+1,L,L))

    U[0],D[0],V[0] = LA.svd(np.eye(L))
    for i in range(Ntau):
        U[i+1], D[i+1], V[i+1] = SVD_stablizer(ExpM(phi[i], ExpT, Vdiag),U[-1],D[-1],V[-1])
    return U,D,V

In [47]:
phi = np.random.choice([1, -1], size=(Ntau, L))
#part of spin up and spin down are not coupled but direct product
Uup,Dup,Vup = Stable_Bmatrix(phi,ExpT,Vdiag)
Udn,Ddn,Vdn = Stable_Bmatrix(phi,ExpT,-Vdiag)

In [51]:
def Update_HS(GFup, GFdn, phi):
    for i in range(phi):
        dup = np.exp(2 * alpha * phi[i]) -1 
        ddn = np.exp(-2 * alpha * phi[i]) -1
        R = (1 + dup * (1 - GFup[i,i]))*(1 + ddn * (1 - GFdn[i,i]))
        if  R > np.random.rand():
            phi[i] = -phi[i]
            GFup -= dup * np.outer(GFup[:,i], 1-GFup[i,:])/R
            GFdn -= ddn * np.outer(GFdn[:,i], 1-GFdn[i,:])/R
        
    return GFup, GFdn, phi

In [ ]:
def Sweep(Uup,Dup,Vup,Udn,Ddn,Vdn,phi,dir,Nstable):
    # start from back sweep and then forward sweep

    # Include B(0,0) and B(beta,beta)
    Ntau = len(phi)+1 
    GFup = np.zeros((Ntau,L,L))
    GFdn = np.zeros((Ntau,L,L))

    if dir = 'Backward':
        indice = range(Ntau-1,-1,-1)
    elif dir = 'Forward':
        indice = range(Ntau)
    else:
        raise ValueError('dir must be Backward or Forward')
    
    for i in range(Ntau):
        if np.mod(i,Nstable):
            # recompute equal time GF to avoid accumulation of numerical error

            
            # compare with advanced one to get the accumulated error
        
        # Update H-S field and Green function
        GFup[i],GFdn[i],phi[i] = Update_HS(GFup[i],GFdn[i],phi[i])

        # Update SVD decompostion for B matrix
        Bup = 

        # Advance equal time GF using new H-S field
        GFup[i+1] = Bup.dot(GFup[i]).dto(LA.inv(Bup))




In [48]:
A = np.random.rand(4,4)

In [50]:
A[:,1]

array([0.51459224, 0.34167914, 0.21885183, 0.95754502])